# Librerias necesarias

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.linear_model import (
    LinearRegression,
    Ridge,
    Lasso,
    ElasticNet,
    RidgeCV,
    ElasticNetCV,
    LassoCV,
    SGDRegressor,
    LogisticRegression
)
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import (
     mean_squared_error, 
     r2_score, 
     mean_absolute_error,
     classification_report, 
     confusion_matrix,
     ConfusionMatrixDisplay,
     balanced_accuracy_score, 
     log_loss,
     roc_curve, 
     roc_auc_score, 
     auc,
     accuracy_score
)
import shap

# Carga de datos

In [ ]:
### Carga datos de dataset
### Contiene aproximadamente 10 años de observaciones diarias de variables climáticas: temperatura, dirección y velocidad del viento, humedad, presión, nubosidad, y cantidad de lluvia en mm.
### tras observar los datos del día de hoy, el objetivo es predecir las variables target:
###                                                                                     -RainFallTomorrow: cantidad de lluvia del día posterior a la observación. Problema de Regresión.
###                                                                                     -RainTomorrow: si el día siguiente llueve o no llueve. Problema de Clasificación.
file_path = "weatherAUS.csv"
df = pd.read_csv(file_path, sep=",", engine="python")

# Limpieza y transformacion de datos

Elimino la columna 'Unnamed: 0' porque es un indice que esta de mas.


In [ ]:
df = df.drop("Unnamed: 0", axis=1)

Segun el enunciado, unicamente nos interesan las ciudades Adelaide, Canberra, Cobar, Dartmoor, Melbourne, MelbourneAirport, MountGambier, Sydney y SydneyAirport por lo que filtro el DataSet para quedarme unicamente con los datos de dichas ciudades.

Tambien elimino de una vez la variable 'Location' debido a que el enunicado declara que se pueden considerar como una unica ubicacion.


In [ ]:
ciudades = [
    " Adelaide",
    "Canberra",
    "Cobar",
    "Dartmoor",
    "Melbourne",
    "MelbourneAirport",
    "MountGambier",
    "Sydney",
    "SydneyAirport",
]
df = df[df["Location"].isin(ciudades)]
df = df.drop("Location", axis=1)

### Split de datos

Hago el split en df_train y df_test a partir de una fecha determinada para dejar aproximadamente un 80% de mis datos en Train y 20% en Test.

In [ ]:
# Convierto la columna 'Date' a tipo datetime
df["Date"] = pd.to_datetime(df["Date"])

In [ ]:
# Fecha más antigua
fecha_mas_antigua = df['Date'].min()
fecha_mas_reciente = df['Date'].max()

print(f'Fecha mas antigua: {fecha_mas_antigua}')
print(f'Fecha mas reciente: {fecha_mas_reciente}')

La fecha mas antigua del dataset es 01-11-2007 y la mas reciente es 24-06-2017. 
Decido hacer el split de datos a partir de la fecha 01-01-2016, concentrando aproximadamente el 80% de datos para el conjunto de entrenamiento, y el 20% restante para el conjunto de test.

In [ ]:
fecha_limite = "2016-01-01"

df_train = df[df["Date"] < fecha_limite]

df_test = df[df["Date"] >= fecha_limite]

print("Tamaño del conjunto de entrenamiento:", len(df_train))
print("Tamaño del conjunto de prueba:", len(df_test))

---


### Tipos de datos y valores nulos

Observo una descripcion, el tipo de dato y los valores nulos de cada variable.


In [ ]:
df_train.describe()

In [ ]:
df_train.dtypes

In [ ]:
df_train.isnull().sum()

Observo que las variables 'RainToday', 'RainTomorrow' y 'RainfallTomorrow' tienen igual cantidad de valores nulos.

Me fijo en que registros las tres columnas son nulas, son unicamente 570 registros, lo que representa aproximadamente un 2% de mi dataset, por lo que decido eliminarlos.


In [ ]:
# Registros de df_train donde las 3 variables son Nulas.
df_train[
    df_train["RainToday"].isnull()
    & df_train["RainTomorrow"].isnull()
    & df_train["RainfallTomorrow"].isnull()
]

In [ ]:
df_train = df_train[
    ~(
        df_train["RainToday"].isnull()
        & df_train["RainTomorrow"].isnull()
        & df_train["RainfallTomorrow"].isnull()
    )
]

In [ ]:
print("Nulos RainToday:", df_train["RainToday"].isnull().sum())
print("Nulos RainTomorrow:", df_train["RainTomorrow"].isnull().sum())
print("Nulos RainfallTomorrow:", df_train["RainfallTomorrow"].isnull().sum())

En cada variable quedaron un total de 162 nulos, vuelvo a observar pero esta vez de a pares, en que variables hay nulos a la vez


In [ ]:
# Observo de a pares df_train
print(
    "Nulos RainToday y RainTomorrow:",
    (df_train["RainToday"].isnull() & df_train["RainTomorrow"].isnull()).sum(),
)
print(
    "Nulos RainToday y RainfallTomorrow:",
    (df_train["RainToday"].isnull() & df_train["RainfallTomorrow"].isnull()).sum(),
)
print(
    "Nulos RainTomorrow y RainfallTomorrow:",
    (df_train["RainTomorrow"].isnull() & df_train["RainfallTomorrow"].isnull()).sum(),
)

Procedo a eliminar los registros nulos de las variables 'RainTomorrow' y 'RainfallTomorrow'.


In [ ]:
df_train[df_train["RainTomorrow"].isnull() & df_train["RainfallTomorrow"].isnull()]

In [ ]:
df_train = df_train[
    ~(df_train["RainTomorrow"].isnull() & df_train["RainfallTomorrow"].isnull())
]

### Matriz de correlacion

In [ ]:
plt.figure(figsize=(15, 15))
sns.heatmap(
    df_train[
        [
            "MinTemp",
            "MaxTemp",
            "Rainfall",
            "Evaporation",
            "Sunshine",
            "WindGustSpeed",
            "WindSpeed9am",
            "WindSpeed3pm",
            "Humidity9am",
            "Humidity3pm",
            "Pressure9am",
            "Pressure3pm",
            "Cloud9am",
            "Cloud3pm",
            "Temp9am",
            "Temp3pm",
            "RainfallTomorrow",
        ]
    ].corr(),
    annot=True,
)
plt.show()

Para rellenar valores nulos, decidi agregar una columna a mi df donde especifico el **Bimestre del año** al que pertenece cada registro, esto lo hago para tener de alguna manera los datos mas segmentados y no calcular una Media, Mediana, o lo que corresponda sobre todos los datos juntos ya que por ejemplo, las temperaturas, vientos, lluvias, etc. pueden no ser lo mismo al inicio del año como por la mitad o al final.


In [ ]:
def determinar_bimestre(fecha):
    mes = fecha.month
    if 1 <= mes <= 2:
        return "Bimestre 1"
    elif 3 <= mes <= 4:
        return "Bimestre 2"
    elif 5 <= mes <= 6:
        return "Bimestre 3"
    elif 7 <= mes <= 8:
        return "Bimestre 4"
    elif 9 <= mes <= 10:
        return "Bimestre 5"
    else:
        return "Bimestre 6"

In [ ]:
df_train["Bimestre"] = df_train["Date"].apply(lambda x: determinar_bimestre(x))

df_test["Bimestre"] = df_test["Date"].apply(lambda x: determinar_bimestre(x))

### Variable: Rainfall


In [ ]:
print(df_train["Rainfall"].isnull().sum())

In [ ]:
# Diagrama de densidades de la variable RainFall
bimestre_1 = df_train[df_train["Bimestre"] == "Bimestre 1"]
bimestre_2 = df_train[df_train["Bimestre"] == "Bimestre 2"]
bimestre_3 = df_train[df_train["Bimestre"] == "Bimestre 3"]
bimestre_4 = df_train[df_train["Bimestre"] == "Bimestre 4"]
bimestre_5 = df_train[df_train["Bimestre"] == "Bimestre 5"]
bimestre_6 = df_train[df_train["Bimestre"] == "Bimestre 6"]


bandwidth = 0.5
sns.kdeplot(
    data=bimestre_1["Rainfall"], fill=True, label="bimestre_1", bw_adjust=bandwidth
)
sns.kdeplot(
    data=bimestre_2["Rainfall"], fill=True, label="bimestre_2", bw_adjust=bandwidth
)
sns.kdeplot(
    data=bimestre_3["Rainfall"], fill=True, label="bimestre_3", bw_adjust=bandwidth
)
sns.kdeplot(
    data=bimestre_4["Rainfall"], fill=True, label="bimestre_4", bw_adjust=bandwidth
)
sns.kdeplot(
    data=bimestre_5["Rainfall"], fill=True, label="bimestre_5", bw_adjust=bandwidth
)
sns.kdeplot(
    data=bimestre_6["Rainfall"], fill=True, label="bimestre_6", bw_adjust=bandwidth
)
plt.title("Diagrama de Densidad de lluvias por Bimestre")
plt.xlabel("Rainfall")
plt.legend()

Puedo apreciar que la densidad de lluvia no depende del bimestre, por lo que, en este caso, no haria falta hacer esta diferenciacion.


In [ ]:
plt.figure(figsize=(8, 6))
sns.boxplot(x="Rainfall", data=df_train)
plt.title("Boxplot de Rainfall")
plt.show()

Veo una gran presencia de valores Outliers por lo que me inclino a usar la Mediana como medida para rellenar los valores nulos de la variable RainFall


In [ ]:
mediana_por_dia_train = df_train.groupby(df["Date"].dt.date)["Rainfall"].median()

df_train["Rainfall"] = df_train.apply(
    lambda row: (
        mediana_por_dia_train[row["Date"].date()]
        if pd.isnull(row["Rainfall"])
        else row["Rainfall"]
    ),
    axis=1,
)

mediana_por_dia_test = df_test.groupby(df["Date"].dt.date)["Rainfall"].median()

df_test["Rainfall"] = df_test.apply(
    lambda row: (
        mediana_por_dia_test[row["Date"].date()]
        if pd.isnull(row["Rainfall"])
        else row["Rainfall"]
    ),
    axis=1,
)

##### Variable: Evaporation


In [ ]:
print(df_train["Evaporation"].isnull().sum())

In [ ]:
plt.figure(figsize=(8, 6))
sns.boxplot(x="Evaporation", data=df_train, color="green")
plt.title("Boxplot de Evaporation")
# plt.ylabel("MaxTemp")
plt.show()

Veo una gran presencia de valores Outliers por lo que me inclino a usar la Mediana como medida para rellenar los valores nulos de la variable **Evaporation**


In [ ]:
# Diagrama de densidades de la variable Evaporation
bimestre_1 = df_train[df_train["Bimestre"] == "Bimestre 1"]
bimestre_2 = df_train[df_train["Bimestre"] == "Bimestre 2"]
bimestre_3 = df_train[df_train["Bimestre"] == "Bimestre 3"]
bimestre_4 = df_train[df_train["Bimestre"] == "Bimestre 4"]
bimestre_5 = df_train[df_train["Bimestre"] == "Bimestre 5"]
bimestre_6 = df_train[df_train["Bimestre"] == "Bimestre 6"]


bandwidth = 0.5
sns.kdeplot(
    data=bimestre_1["Evaporation"], fill=True, label="bimestre_1", bw_adjust=bandwidth
)
sns.kdeplot(
    data=bimestre_2["Evaporation"], fill=True, label="bimestre_2", bw_adjust=bandwidth
)
sns.kdeplot(
    data=bimestre_3["Evaporation"], fill=True, label="bimestre_3", bw_adjust=bandwidth
)
sns.kdeplot(
    data=bimestre_4["Evaporation"], fill=True, label="bimestre_4", bw_adjust=bandwidth
)
sns.kdeplot(
    data=bimestre_5["Evaporation"], fill=True, label="bimestre_5", bw_adjust=bandwidth
)
sns.kdeplot(
    data=bimestre_6["Evaporation"], fill=True, label="bimestre_6", bw_adjust=bandwidth
)
plt.title("Diagrama de Densidad de Evaporation por Bimestre")
plt.xlabel("Evaporation")
plt.legend()

En este caso se puede apreciar una variacion en la densidad de la variable Evaporation respecto del bimestre.


In [ ]:
bim = df_train.groupby("Bimestre")
medians = bim["Evaporation"].median()

for bimestre, median in medians.items():
    df_train.loc[
        (df_train["Bimestre"] == bimestre) & (df_train["Evaporation"].isnull()),
        "Evaporation",
    ] = median


for bimestre, median in medians.items():
    df_test.loc[
        (df_test["Bimestre"] == bimestre) & (df_test["Evaporation"].isnull()),
        "Evaporation",
    ] = median

##### Variable: Sunshine


In [ ]:
print(df_train["Sunshine"].isnull().sum())

In [ ]:
plt.figure(figsize=(8, 6))
sns.boxplot(x="Sunshine", data=df_train, color="orange")
plt.title("Boxplot de Sunshine")
plt.show()

La distribucion de la variable Sunshine se ve bastante balanceada y sin presencia de outliers por lo que utilizo la **Media** para imputar a los valores nulos.


In [ ]:
df_train["Sunshine"] = df_train.groupby(df_train["Date"].dt.day)["Sunshine"].transform(
    lambda x: x.fillna(x.mean())
)

df_test["Sunshine"] = df_test.groupby(df_test["Date"].dt.day)["Sunshine"].transform(
    lambda x: x.fillna(x.mean())
)

##### Variables: WindGustDir, WindDir9am y WindDir3pm


In [ ]:
print(df_train["WindGustDir"].isnull().sum())
print(df_train["WindDir9am"].isnull().sum())
print(df_train["WindDir3pm"].isnull().sum())

Relleno los valores faltantes para cada variable utilizando la **Moda** de cada dia.


In [ ]:
df_train["WindGustDir"] = df_train.groupby(df_train["Date"].dt.day)[
    "WindGustDir"
].transform(lambda x: x.fillna(x.mode().iloc[0]))
df_train["WindDir9am"] = df_train.groupby(df_train["Date"].dt.day)[
    "WindDir9am"
].transform(lambda x: x.fillna(x.mode().iloc[0]))
df_train["WindDir3pm"] = df_train.groupby(df_train["Date"].dt.day)[
    "WindDir3pm"
].transform(lambda x: x.fillna(x.mode().iloc[0]))


df_test["WindGustDir"] = df_test.groupby(df_test["Date"].dt.day)[
    "WindGustDir"
].transform(lambda x: x.fillna(x.mode().iloc[0]))
df_test["WindDir9am"] = df_test.groupby(df_test["Date"].dt.day)["WindDir9am"].transform(
    lambda x: x.fillna(x.mode().iloc[0])
)
df_test["WindDir3pm"] = df_test.groupby(df_test["Date"].dt.day)["WindDir3pm"].transform(
    lambda x: x.fillna(x.mode().iloc[0])
)

##### Variables: WindGustSpeed, WindSpeed9am y WindSpeed3pm


In [ ]:
data_to_plot = df_train[["WindGustSpeed", "WindSpeed9am", "WindSpeed3pm"]]

plt.figure(figsize=(10, 6))
sns.boxplot(data=data_to_plot, palette="Set3")
plt.title("Boxplots de WindSpeed9am, WindSpeed3pm y WindGustSpeed")
plt.ylabel("Speed")
plt.show()

Se observa una gran presencia de outlaiers en las 3 variables por lo que procedo a imputar los valores nulos utilizando la **Mediana**.


In [ ]:
df_train["WindGustSpeed"] = df_train.groupby(df_train["Date"].dt.day)[
    "WindGustSpeed"
].transform(lambda x: x.fillna(x.median()))
df_train["WindSpeed9am"] = df_train.groupby(df_train["Date"].dt.day)[
    "WindSpeed9am"
].transform(lambda x: x.fillna(x.median()))
df_train["WindSpeed3pm"] = df_train.groupby(df_train["Date"].dt.day)[
    "WindGustSpeed"
].transform(lambda x: x.fillna(x.median()))


df_test["WindGustSpeed"] = df_test.groupby(df_test["Date"].dt.day)[
    "WindGustSpeed"
].transform(lambda x: x.fillna(x.median()))
df_test["WindSpeed9am"] = df_test.groupby(df_test["Date"].dt.day)[
    "WindSpeed9am"
].transform(lambda x: x.fillna(x.median()))
df_test["WindSpeed3pm"] = df_test.groupby(df_test["Date"].dt.day)[
    "WindGustSpeed"
].transform(lambda x: x.fillna(x.median()))

Genero una nueva columna llamada 'Dif_WindSpeed' imputandole el valor correspondiente a la diferencia de las columnas 'WindSpeed9am' y 'WindSpeed3pm' **( 'WindSpeed9am' - 'WindSpeed3pm' )**


In [ ]:
df_train["WindSpeed_Difference"] = df_train["WindSpeed9am"] - df_train["WindSpeed3pm"]
df_train.drop(["WindSpeed9am", "WindSpeed3pm"], axis=1, inplace=True)


df_test["WindSpeed_Difference"] = df_test["WindSpeed9am"] - df_test["WindSpeed3pm"]
df_test.drop(["WindSpeed9am", "WindSpeed3pm"], axis=1, inplace=True)

##### Variables: Humidity9am, Humidity3pm, Cloud9am, Cloud3pm, Pressure9am y Pressure3pm


In [ ]:
print(df_train["Humidity9am"].isnull().sum())
print(df_train["Humidity3pm"].isnull().sum())
print(df_train["Cloud9am"].isnull().sum())
print(df_train["Cloud3pm"].isnull().sum())
print(df_train["Pressure9am"].isnull().sum())
print(df_train["Pressure3pm"].isnull().sum())

In [ ]:
data_to_plot = df_train[["Humidity9am", "Humidity3pm"]]
plt.figure(figsize=(10, 6))
sns.boxplot(data=data_to_plot)
plt.title("Boxplots de Humidity9am y Humidity3pm")
plt.show()

In [ ]:
df_train["Humidity9am"] = df_train.groupby(df_train["Date"].dt.day)[
    "Humidity9am"
].transform(lambda x: x.fillna(x.median()))
df_train["Humidity3pm"] = df_train.groupby(df_train["Date"].dt.day)[
    "Humidity3pm"
].transform(lambda x: x.fillna(x.median()))




df_test["Humidity9am"] = df_test.groupby(df_test["Date"].dt.day)[
    "Humidity9am"
].transform(lambda x: x.fillna(x.median()))
df_test["Humidity3pm"] = df_test.groupby(df_test["Date"].dt.day)[
    "Humidity3pm"
].transform(lambda x: x.fillna(x.median()))

Genero una nueva columna llamada 'Dif_Humidity' imputandole el valor correspondiente a la diferencia de las columnas 'Humidity9am' y 'HUmidity3pm' **( 'Humidity9am' - 'Humidity3pm' )**


In [ ]:
df_train["Humidity_Difference"] = df_train["Humidity9am"] - df_train["Humidity3pm"]
df_train.drop(["Humidity9am", "Humidity3pm"], axis=1, inplace=True)


df_test["Humidity_Difference"] = df_test["Humidity9am"] - df_test["Humidity3pm"]
df_test.drop(["Humidity9am", "Humidity3pm"], axis=1, inplace=True)

In [ ]:
plt.hist(df_train["Cloud9am"], bins=20, color="skyblue", alpha=0.7, label="Cloud9am")
plt.hist(df_train["Cloud3pm"], bins=20, color="salmon", alpha=0.7, label="Cloud3pm")
plt.xlabel("Nubosidad")
plt.ylabel("Frecuencia")
plt.title("Histograma de Cloud9am y Cloud3pm")
plt.legend()
plt.show()

In [ ]:
df_train["Cloud9am"] = df_train.groupby(df_train["Date"].dt.day)["Cloud9am"].transform(
    lambda x: x.fillna(x.median())
)
df_train["Cloud3pm"] = df_train.groupby(df_train["Date"].dt.day)["Cloud3pm"].transform(
    lambda x: x.fillna(x.median())
)


df_test["Cloud9am"] = df_test.groupby(df_test["Date"].dt.day)["Cloud9am"].transform(
    lambda x: x.fillna(x.median())
)
df_test["Cloud3pm"] = df_test.groupby(df_test["Date"].dt.day)["Cloud3pm"].transform(
    lambda x: x.fillna(x.median())
)

Genero una nueva columna llamada 'Dif_Cloud' imputandole el valor correspondiente a la diferencia de las columnas 'Humidity9am' y 'HUmidity3pm' **( 'Cloud9am' - 'Cloud3pm' )**


In [ ]:
df_train["Cloud_Difference"] = df_train["Cloud9am"] - df_train["Cloud3pm"]
df_train.drop(["Cloud9am", "Cloud3pm"], axis=1, inplace=True)


df_test["Cloud_Difference"] = df_test["Cloud9am"] - df_test["Cloud3pm"]
df_test.drop(["Cloud9am", "Cloud3pm"], axis=1, inplace=True)

In [ ]:
sns.kdeplot(df_train["Pressure9am"], color="skyblue", label="Pressure9am", fill=True)
sns.kdeplot(df_train["Pressure3pm"], color="salmon", label="Pressure3pm", fill=True)
plt.xlabel("Pressure")
plt.ylabel("Densidad")
plt.title("Gráfico de Densidad de Pressure9am y Pressure3pm")
plt.legend()
plt.show()

Observando la distribucion de las variables 'Pressure9am' y 'Pressure3pm' se observa una distribucion normal, por lo que decido imputar los valores nulos utilizando la **Media**.


In [ ]:
df_train["Pressure9am"] = df_train.groupby(df_train["Date"].dt.day)[
    "Pressure9am"
].transform(lambda x: x.fillna(x.mean()))
df_train["Pressure3pm"] = df_train.groupby(df_train["Date"].dt.day)[
    "Pressure3pm"
].transform(lambda x: x.fillna(x.mean()))


df_test["Pressure9am"] = df_test.groupby(df_test["Date"].dt.day)[
    "Pressure9am"
].transform(lambda x: x.fillna(x.mean()))
df_test["Pressure3pm"] = df_test.groupby(df_test["Date"].dt.day)[
    "Pressure3pm"
].transform(lambda x: x.fillna(x.mean()))

Genero una nueva columna llamada 'Dif_Pressure' imputandole el valor correspondiente a la diferencia de las columnas 'Pressure9am' y 'Pressure3pm' **( 'Pressure9am' - 'Pressure3pm' )**


In [ ]:
df_train["Pressure_Difference"] = df_train["Pressure9am"] - df_train["Pressure3pm"]
df_train.drop(["Pressure9am", "Pressure3pm"], axis=1, inplace=True)


df_test["Pressure_Difference"] = df_test["Pressure9am"] - df_test["Pressure3pm"]
df_test.drop(["Pressure9am", "Pressure3pm"], axis=1, inplace=True)

##### Variables: MaxTemp, MinTemp, Temp9am y Temp3pm


In [ ]:
# Calcula la media de temperatura mínima para cada bimestre
mean_temps = df_train.groupby("Bimestre")["MinTemp"].mean().reset_index()
plt.figure(figsize=(10, 6))
sns.barplot(
    x="Bimestre",
    y="MinTemp",
    data=mean_temps,
    hue="Bimestre",
    palette="Set1",
    dodge=False,
    legend=False,
)
plt.title("Media de Temperatura Mínima por Bimestre")
plt.xlabel("Bimestre")
plt.ylabel("Temperatura Mínima Media (°C)")
plt.show()

In [ ]:
# Calcula la media de temperatura mínima para cada bimestre
mean_temps = df_train.groupby("Bimestre")["MaxTemp"].mean().reset_index()
plt.figure(figsize=(10, 6))
sns.barplot(
    x="Bimestre",
    y="MaxTemp",
    data=mean_temps,
    hue="Bimestre",
    palette="Set2",
    dodge=False,
    legend=False,
)
plt.title("Media de Temperatura Maxima por Bimestre")
plt.xlabel("Bimestre")
plt.ylabel("Temperatura Mínima Media (°C)")
plt.show()

In [ ]:
data_to_plot = df_train[["MinTemp", "MaxTemp"]]
plt.figure(figsize=(10, 6))
sns.boxplot(data=data_to_plot, palette="Set1")
plt.title("Boxplots de MinTemp y MaxTemp")
plt.show()

Puedo apreciar variaciones en los valores tanto en la variable MinTemp como MaxTemp dependiendo del bimestre del año, a la vez observo outliers por lo que en este caso decido rellenar los valores nulos con la Mediana del bimestre correspondiente a cada registro.


In [ ]:
median_min_temp_by_bimestre_train = df_train.groupby("Bimestre")["MinTemp"].median()

for bimestre, median_temp in median_min_temp_by_bimestre_train.items():
    df_train.loc[df_train["Bimestre"] == bimestre, "MinTemp"] = df_train.loc[
        df_train["Bimestre"] == bimestre, "MinTemp"
    ].fillna(median_temp)

for bimestre, median_temp in median_min_temp_by_bimestre_train.items():
    df_test.loc[df_test["Bimestre"] == bimestre, "MinTemp"] = df_test.loc[
        df_test["Bimestre"] == bimestre, "MinTemp"
    ].fillna(median_temp)


median_max_temp_by_bimestre_train = df_train.groupby("Bimestre")["MaxTemp"].median()

for bimestre, median_temp in median_max_temp_by_bimestre_train.items():
    df_train.loc[df_train["Bimestre"] == bimestre, "MaxTemp"] = df_train.loc[
        df_train["Bimestre"] == bimestre, "MaxTemp"
    ].fillna(median_temp)

for bimestre, median_temp in median_max_temp_by_bimestre_train.items():
    df_test.loc[df_test["Bimestre"] == bimestre, "MaxTemp"] = df_test.loc[
        df_test["Bimestre"] == bimestre, "MaxTemp"
    ].fillna(median_temp)

In [ ]:
data_to_plot = df_train[["Temp9am", "Temp3pm"]]
plt.figure(figsize=(10, 6))
sns.boxplot(data=data_to_plot, palette="Set2")
plt.title("Boxplots de Temp9am y Temp3pm")
plt.show()

Completo los valores nulos de Temp9am y Temp3pm con la mediana por dia debido a la presencia de outliers.


In [ ]:
df_train["Temp9am"] = df_train.groupby(df_train["Date"].dt.day)["Temp9am"].transform(
    lambda x: x.fillna(x.median())
)
df_train["Temp3pm"] = df_train.groupby(df_train["Date"].dt.day)["Temp3pm"].transform(
    lambda x: x.fillna(x.median())
)

df_test["Temp9am"] = df_test.groupby(df_test["Date"].dt.day)["Temp9am"].transform(
    lambda x: x.fillna(x.median())
)
df_test["Temp3pm"] = df_test.groupby(df_test["Date"].dt.day)["Temp3pm"].transform(
    lambda x: x.fillna(x.median())
)

In [ ]:
df_train.isna().sum()

Genero una nueva columna llamada 'Dif_Temp' imputandole el valor correspondiente a la diferencia de las columnas 'Temp3pm' y 'Temp9am' **( 'Temp3pm' - 'Temp9am' )**


In [ ]:
df_train["Temp_Difference"] = df_train["Temp3pm"] - df_train["Temp9am"]
df_train.drop(["Temp3pm", "Temp9am"], axis=1, inplace=True)

df_test["Temp_Difference"] = df_test["Temp3pm"] - df_test["Temp9am"]
df_test.drop(["Temp3pm", "Temp9am"], axis=1, inplace=True)

Genero una nueva columna llamada 'Dif_Temp_Max_Min' imputandole el valor correspondiente a la diferencia de las columnas 'MaxTemp' y 'MinTemp' **( 'MaxTemp' - 'MinTemp' )**


In [ ]:
df_train["Dif_Temp_Max_Min"] = df_train["MaxTemp"] - df_train["MinTemp"]
df_train.drop(["MaxTemp", "MinTemp"], axis=1, inplace=True)

df_test["Dif_Temp_Max_Min"] = df_test["MaxTemp"] - df_test["MinTemp"]
df_test.drop(["MaxTemp", "MinTemp"], axis=1, inplace=True)

##### Variables: RainToday y RainTomorrow


Relleno los valores nulos de 'RainToday' con la **Moda** por dia.


In [ ]:
print(df_train["RainToday"].isna().sum())
print(df_train["RainTomorrow"].isna().sum())
print(df_train["RainfallTomorrow"].isna().sum())

In [ ]:
moda_RainToday_train = df_train.groupby("Date")["RainToday"].transform(
    lambda x: x.mode().iloc[0] if not x.mode().empty else None
)
df_train["RainToday"] = df_train["RainToday"].fillna(moda_RainToday_train)

moda_RainToday_test = df_test.groupby("Date")["RainToday"].transform(
    lambda x: x.mode().iloc[0] if not x.mode().empty else None
)
df_test["RainToday"] = df_test["RainToday"].fillna(moda_RainToday_test)


moda_RainTomorrow_test = df_test.groupby("Date")["RainTomorrow"].transform(
    lambda x: x.mode().iloc[0] if not x.mode().empty else None
)
df_test["RainTomorrow"] = df_test["RainTomorrow"].fillna(moda_RainTomorrow_test)

moda_RainfallTomorrow_test = df_test.groupby("Date")["RainfallTomorrow"].transform(
    lambda x: x.mode().iloc[0] if not x.mode().empty else None
)
df_test["RainfallTomorrow"] = df_test["RainfallTomorrow"].fillna(
    moda_RainfallTomorrow_test
)

In [ ]:
df_test.isna().sum()

In [ ]:
print(df_train["RainToday"].isna().sum())
print(df_train["RainTomorrow"].isna().sum())
print(df_train["RainfallTomorrow"].isna().sum())

La columna 'RainToday' y 'RainTomorrow' tienen valores 'Yes' 'No' por lo que los mapeo a 1 para 'Yes' y 0 para 'No'.


In [ ]:
# df_train
df_train["RainToday"] = df_train["RainToday"].map({"Yes": 1, "No": 0})
df_train["RainTomorrow"] = df_train["RainTomorrow"].map({"Yes": 1, "No": 0})
# df_test
df_test["RainToday"] = df_test["RainToday"].map({"Yes": 1, "No": 0})
df_test["RainTomorrow"] = df_test["RainTomorrow"].map({"Yes": 1, "No": 0})

In [ ]:
df_train.isna().sum()

### Dummies


In [ ]:
df_train["WindGustDir"].unique()

In [ ]:
orien = [
    "SSW",
    "S",
    "SE",
    "NNE",
    "WNW",
    "N",
    "ENE",
    "NE",
    "E",
    "SW",
    "W",
    "WSW",
    "NNW",
    "ESE",
    "SSE",
    "NW",
]
print(len(orien))

Agrupo los valores de la variables categoricas de Direccion, que tiene los siguientes valores:

['SSW', 'S', 'SE', 'NNE', 'WNW', 'N', 'ENE', 'NE', 'E', 'SW', 'W', 'WSW', 'NNW', 'ESE', 'SSE', 'NW']

El criterio que empleo es asignar a cada punto cardenal el predominante, por ejemplo 'NNW' lo asigno a 'N'

Para los valores como, por ejemplo 'NE' o 'SW' los asigno al ultimo punto cardinal de la notacion.


In [ ]:
def agrupar_direcciones(direccion):
    grupos_principales = {
        "N": ["N", "NNW", "NNE"],
        "S": ["S", "SSW", "SSE"],
        "E": ["E", "ENE", "ESE", "SE", "NE"],
        "W": ["W", "WNW", "WSW", "SW", "NW"],
    }

    for grupo, direcciones in grupos_principales.items():
        if direccion in direcciones:
            return grupo

    return "Otro"

In [ ]:
df_train["WindGustDir_Agrupado"] = df_train["WindGustDir"].apply(agrupar_direcciones)
df_train["WindDir9am_Agrupado"] = df_train["WindDir9am"].apply(agrupar_direcciones)
df_train["WindDir3pm_Agrupado"] = df_train["WindDir3pm"].apply(agrupar_direcciones)

df_test["WindGustDir_Agrupado"] = df_test["WindGustDir"].apply(agrupar_direcciones)
df_test["WindDir9am_Agrupado"] = df_test["WindDir9am"].apply(agrupar_direcciones)
df_test["WindDir3pm_Agrupado"] = df_test["WindDir3pm"].apply(agrupar_direcciones)


df_train = df_train.drop("WindGustDir", axis=1)
df_train = df_train.drop("WindDir9am", axis=1)
df_train = df_train.drop("WindDir3pm", axis=1)

df_test = df_test.drop("WindGustDir", axis=1)
df_test = df_test.drop("WindDir9am", axis=1)
df_test = df_test.drop("WindDir3pm", axis=1)

##### Dummies WindGustDir


In [ ]:
d_WindGustDir_train = pd.get_dummies(
    df_train["WindGustDir_Agrupado"], dtype=int, drop_first=True
)

d_WindGustDir_train = d_WindGustDir_train.rename(
    columns={"N": "WindGustDir_N", "S": "WindGustDir_S", "W": "WindGustDir_W"}
)
df_train = df_train.drop("WindGustDir_Agrupado", axis=1)
df_train = pd.concat([df_train, d_WindGustDir_train], axis=1)


d_WindGustDir_test = pd.get_dummies(
    df_test["WindGustDir_Agrupado"], dtype=int, drop_first=True
)

d_WindGustDir_test = d_WindGustDir_test.rename(
    columns={"N": "WindGustDir_N", "S": "WindGustDir_S", "W": "WindGustDir_W"}
)
df_test = df_test.drop("WindGustDir_Agrupado", axis=1)
df_test = pd.concat([df_test, d_WindGustDir_test], axis=1)

##### Dummies WindDir9am


In [ ]:
d_WindDir9am_train = pd.get_dummies(
    df_train["WindDir9am_Agrupado"], dtype=int, drop_first=True
)
d_WindDir9am_train = d_WindDir9am_train.rename(
    columns={"N": "WindDir9am_N", "S": "WindDir9am_S", "W": "WindDir9am_W"}
)
df_train = df_train.drop("WindDir9am_Agrupado", axis=1)
df_train = pd.concat([df_train, d_WindDir9am_train], axis=1)


d_WindDir9am_test = pd.get_dummies(
    df_test["WindDir9am_Agrupado"], dtype=int, drop_first=True
)
d_WindDir9am_test = d_WindDir9am_test.rename(
    columns={"N": "WindDir9am_N", "S": "WindDir9am_S", "W": "WindDir9am_W"}
)
df_test = df_test.drop("WindDir9am_Agrupado", axis=1)
df_test = pd.concat([df_test, d_WindDir9am_test], axis=1)

##### Dummies WindDir3pm


In [ ]:
d_WindDir3pm_train = pd.get_dummies(
    df_train["WindDir3pm_Agrupado"], dtype=int, drop_first=True
)
d_WindDir3pm_train = d_WindDir3pm_train.rename(
    columns={"N": "WindDir3pm_N", "S": "WindDir3pm_S", "W": "WindDir3pm_W"}
)
df_train = df_train.drop("WindDir3pm_Agrupado", axis=1)
df_train = pd.concat([df_train, d_WindDir3pm_train], axis=1)


d_WindDir3pm_test = pd.get_dummies(
    df_test["WindDir3pm_Agrupado"], dtype=int, drop_first=True
)
d_WindDir3pm_test = d_WindDir3pm_test.rename(
    columns={"N": "WindDir3pm_N", "S": "WindDir3pm_S", "W": "WindDir3pm_W"}
)
df_test = df_test.drop("WindDir3pm_Agrupado", axis=1)
df_test = pd.concat([df_test, d_WindDir3pm_test], axis=1)

In [ ]:
df_train.drop("Bimestre", axis=1, inplace=True)
df_train.drop("Date", axis=1, inplace=True)

df_test.drop("Bimestre", axis=1, inplace=True)
df_test.drop("Date", axis=1, inplace=True)

In [ ]:
columnas = df_train.columns
print(columnas)
print(len(columnas))

In [ ]:
plt.figure(figsize=(15, 15))
sns.heatmap(
    df_train.corr(),
    annot=True,
)
plt.show()

In [ ]:
df_train.isna().sum()

# Estandarizacion de datos


In [ ]:
# df_train
scaler = StandardScaler()
df_train_estandarizado = scaler.fit_transform(df_train)
df_train_estandarizado = pd.DataFrame(df_train_estandarizado, columns=df_train.columns)

# df_test
scaler = StandardScaler()
df_test_estandarizado = scaler.fit_transform(df_test)
df_test_estandarizado = pd.DataFrame(df_test_estandarizado, columns=df_test.columns)

# Regularizacion de datos

In [ ]:
# X, y TRAIN
X_train = df_train_estandarizado[
    [
        "Rainfall",
        "Evaporation",
        "Sunshine",
        "WindGustSpeed",
        "RainToday",
        "WindSpeed_Difference",
        "Humidity_Difference",
        "Cloud_Difference",
        "Pressure_Difference",
        "Temp_Difference",
        "Dif_Temp_Max_Min",
        "WindGustDir_N",
        "WindGustDir_S",
        "WindGustDir_W",
        "WindDir9am_N",
        "WindDir9am_S",
        "WindDir9am_W",
        "WindDir3pm_N",
        "WindDir3pm_S",
        "WindDir3pm_W",
    ]
]
y_train = df_train_estandarizado["RainfallTomorrow"]

# X, y TEST

X_test = df_test_estandarizado[
    [
        "Rainfall",
        "Evaporation",
        "Sunshine",
        "WindGustSpeed",
        "RainToday",
        "WindSpeed_Difference",
        "Humidity_Difference",
        "Cloud_Difference",
        "Pressure_Difference",
        "Temp_Difference",
        "Dif_Temp_Max_Min",
        "WindGustDir_N",
        "WindGustDir_S",
        "WindGustDir_W",
        "WindDir9am_N",
        "WindDir9am_S",
        "WindDir9am_W",
        "WindDir3pm_N",
        "WindDir3pm_S",
        "WindDir3pm_W",
    ]
]
y_test = df_test_estandarizado["RainfallTomorrow"]

# Lasso


In [ ]:
lasso = Lasso(alpha=0.1)  # alpha controla la fuerza de la regularización L1 (Lasso)

lasso.fit(X_train, y_train)

In [ ]:
print("\nCoeficientes del modelo Lasso:")
print(lasso.coef_)
print("Lasso Score df_train:", lasso.score(X_test, y_test))
print("Lasso Score df_test:", lasso.score(X_test, y_test))

# Ridge


In [ ]:
ridge = Ridge(alpha=0.1)  # alpha controla la fuerza de la regularización L2 (Ridge)
ridge.fit(X_train, y_train)

In [ ]:
print("\nCoeficientes del modelo Ridge:")
print(ridge.coef_)
print("Ridge Score df_train:", ridge.score(X_train, y_train))
print("Ridge Score df_test:", ridge.score(X_test, y_test))

# Elasticnet

In [ ]:
elasticnet = ElasticNet(alpha=0.1, l1_ratio=0.5)
elasticnet.fit(X_train,y_train)

In [ ]:
print("\nCoeficientes del modelo ElasticNet:")
print(elasticnet.coef_)
print("Elasticnet Score df_train:", elasticnet.score(X_train, y_train))
print("Elasticnet Score df_test:", elasticnet.score(X_test, y_test))


# Regresion Lineal


Para evitar una Fuga de Datos voy a eliminar de mi dataset las variables RainTomorrow y RainfallTomorrow.


In [ ]:
model = LinearRegression()

model.fit(X_train, y_train)

y_pred = model.predict(X_test)

coefficients = model.coef_
intercept = model.intercept_
print("Coefficients:", coefficients)
print("Intercept:", intercept)

# MSE: Error Cuadratico Medio
mse = mean_squared_error(y_test, y_pred)

# R^2
r2 = r2_score(y_test, y_pred)

# MAE: Error Absoluto Medio
mae = mean_absolute_error(y_test, y_pred)

# RMSE: Raíz del Error Cuadrático Medio
rmse = np.sqrt(mean_squared_error(y_test, y_pred))

print(f"\nR^2: {r2}\n")
print(f"MSE(Error Cuadratico Medio): {mse}\n")
print(f"MAE(Error Absoluto Medio): {mae}\n")
print(f"RMSE(Raíz del Error Cuadrático Medio): {rmse}\n")

#### Optimizacion de Hiperparametros

In [ ]:
from sklearn.model_selection import GridSearchCV

# Definir la cuadrícula de hiperparámetros a explorar
param_grid = {
    'fit_intercept': [False, True],
    'copy_X': [True, False]
}
# Inicializar la búsqueda en cuadrícula
grid_search = GridSearchCV(LinearRegression(), param_grid, cv=5)

# Realizar la búsqueda en cuadrícula
grid_search.fit(X_train, y_train)
# Obtener los mejores hiperparámetros
best_params = grid_search.best_params_
print("Mejores hiperparámetros:", best_params)

In [ ]:
model2 = LinearRegression(copy_X=True, fit_intercept=False)

model2.fit(X_train, y_train)

y_predopt = model2.predict(X_test)

coefficientsopt = model2.coef_
interceptopt = model2.intercept_
print("Coefficients:", coefficientsopt)
print("Intercept:", interceptopt)

# MSE: Error Cuadratico Medio
mseopt = mean_squared_error(y_test, y_predopt)

# R^2
r2opt= r2_score(y_test, y_predopt)

# MAE: Error Absoluto Medio
maeopt = mean_absolute_error(y_test, y_predopt)

# RMSE: Raíz del Error Cuadrático Medio
rmseopt = np.sqrt(mean_squared_error(y_test, y_predopt))

print(f"\nR^2: {r2opt}\n")
print(f"MSE(Error Cuadratico Medio): {mseopt}\n")
print(f"MAE(Error Absoluto Medio): {maeopt}\n")
print(f"RMSE(Raíz del Error Cuadrático Medio): {rmseopt}\n")

Con la optimizacion de hiperparametros no aprecio variaciones en las metricas.

## Gradiente descendiente

Probe usar el metodo de clase pero no pude correrlo debido a que me presentaba un error y no supe como solucionarlo.

Por otro lado, busque en internet como hacerlo usando la ScikitLearn y lo implemente.

In [ ]:
# Regresión lineal utilizando SGDRegressor
model_sgd = SGDRegressor(max_iter=1000, tol=1e-3, random_state=42)

model_sgd.fit(X_train, y_train)

y_pred_sgd = model_sgd.predict(X_test)

# Coeficientes
coefficients_sgd = model_sgd.coef_
intercept_sgd = model_sgd.intercept_
print("Coefficients (SGD):", coefficients_sgd)
print("Intercept (SGD):", intercept_sgd)

# MSE: Error Cuadrático Medio
mse_sgd = mean_squared_error(y_test, y_pred_sgd)

# R^2
r2_sgd = r2_score(y_test, y_pred_sgd)

# MAE: Error Absoluto Medio
mae_sgd = mean_absolute_error(y_test, y_pred_sgd)

# RMSE: Raíz del Error Cuadrático Medio
rmse_sgd = np.sqrt(mse_sgd)

print(f"\nR^2 (SGD): {r2_sgd}\n")
print(f"MSE (SGD): {mse_sgd}\n")
print(f"MAE (SGD): {mae_sgd}\n")
print(f"RMSE (SGD): {rmse_sgd}\n")

In [ ]:
y_train_gd = y_train.values.reshape(-1, 1)
y_test_gd = y_test.values.reshape(-1, 1)

In [ ]:
def gradient_descent(X_train, y_train, X_test, y_test, lr=0.01, epochs=100):
    """
    shapes:
        X_train = nxm
        y_train = nx1
        X_test = pxm
        y_test = px1
        W = mx1
    """
    n = X_train.shape[0]
    m = X_train.shape[1]

    o = X_test.shape[0]

    # Poner columna de unos a las matrices X
    X_train = np.hstack((np.ones((n, 1)), X_train))
    X_test = np.hstack((np.ones((o, 1)), X_test))


    # Inicializar pesos aleatorios
    W = np.random.randn(m+1).reshape(m+1, 1)

    train_errors = []  # Para almacenar el error de entrenamiento en cada época
    test_errors = []   # Para almacenar el error de prueba en cada época

    for i in range(epochs):
        # Calcular predicción y error de entrenamiento
        prediction_train = np.matmul(X_train, W)
        error_train = y_train - prediction_train
        #print(error_train)
        train_mse = np.mean(error_train ** 2)
        train_errors.append(train_mse)

        # Calcular predicción y error de prueba
        prediction_test = np.matmul(X_test, W)
        error_test = y_test - prediction_test
        test_mse = np.mean(error_test ** 2)
        test_errors.append(test_mse)

        # Calcular el gradiente y actualizar pesos
        grad_sum = np.sum(error_train * X_train, axis=0)
        grad_mul = -2/n * grad_sum  # 1xm
        gradient = np.transpose(grad_mul).reshape(-1, 1)  # mx1

        W = W - (lr * gradient)

    # Graficar errores de entrenamiento y prueba
    # Definir una figura
    plt.figure(figsize=(12, 6))
    # Plotear errores de entrenamiento
    plt.plot(train_errors, label='Error de entrenamiento')
    # Plotear errores de prueba
    plt.plot(test_errors, label='Error de test')
    # Poner labels en los ejes
    plt.xlabel('Época')
    plt.ylabel('Error cuadrático medio')
    # Activar la leyenda
    plt.legend()
    # Poner titulo
    plt.title('Error de entrenamiento y prueba vs iteraciones (GD)')
    # Terminar y mostrar gráfico
    plt.show()

    return W

In [ ]:
gradient_descent(X_train, y_train_gd, X_test, y_test_gd, lr=0.01, epochs=100)

In [ ]:
def stochastic_gradient_descent(X_train, y_train, X_test, y_test, lr=0.01, epochs=100):

    n = X_train.shape[0]
    m = X_train.shape[1]

    X_train = np.hstack((np.ones((n, 1)), X_train))
    X_test = np.hstack((np.ones((X_test.shape[0], 1)), X_test))

    W = np.random.randn(m + 1).reshape(-1, 1)

    train_errors = []
    test_errors = []

    for i in range(epochs):
        # Permutación aleatoria de los datos
        permutation = np.random.permutation(n)
        X_train = X_train[permutation]
        y_train = y_train[permutation]

        for j in range(n):
            # Obtener una muestra aleatoria de un solo dato para hacer SGD
            x_sample = X_train[j]
            y_sample = y_train[j][0]

            prediction = np.matmul(x_sample, W)
            error = y_sample - prediction
            train_mse = error ** 2
            train_errors.append(train_mse)

            gradient = -2 * error * x_sample.T.reshape(-1, 1)

            W = W - (lr * gradient)

            prediction_test = np.matmul(X_test, W)
            error_test = y_test - prediction_test
            test_mse = np.mean(error_test ** 2)
            test_errors.append(test_mse)

    plt.figure(figsize=(12, 6))
    plt.plot(train_errors, label='Error de entrenamiento')
    plt.plot(test_errors, label='Error de prueba')
    plt.xlabel('Iteración')
    plt.ylabel('Error cuadrático medio')
    plt.legend()
    plt.title('Error de entrenamiento y prueba vs iteraciones (SGD)')
    plt.show()

    return W

In [ ]:
stochastic_gradient_descent(X_train, y_train_gd, X_test, y_test_gd, lr=0.01, epochs=100)

In [ ]:
def mini_batch_gradient_descent(X_train, y_train, X_test, y_test, lr=0.01, epochs=100, batch_size=11):
    n = X_train.shape[0]
    m = X_train.shape[1]

    X_train = np.hstack((np.ones((n, 1)), X_train))
    X_test = np.hstack((np.ones((X_test.shape[0], 1)), X_test))

    W = np.random.randn(m + 1).reshape(-1, 1)

    train_errors = []
    test_errors = []

    for i in range(epochs):

        # Permutación aleatoria de los datos
        permutation = np.random.permutation(n)
        X_train = X_train[permutation]
        y_train = y_train[permutation]


        for j in range(0, n, batch_size):
            # Obtener un lote (mini-batch) de datos
            x_batch = X_train[j:j+batch_size, :]
            y_batch = y_train[j:j+batch_size].reshape(-1, 1)

            prediction = np.matmul(x_batch, W)
            error = y_batch - prediction
            train_mse = np.mean(error ** 2)
            train_errors.append(train_mse)

            gradient = -2 * np.matmul(x_batch.T, error) / batch_size

            W = W - (lr * gradient)

            prediction_test = np.matmul(X_test, W)
            error_test = y_test - prediction_test
            test_mse = np.mean(error_test ** 2)
            test_errors.append(test_mse)

    plt.figure(figsize=(12, 6))
    plt.plot(train_errors, label='Error de entrenamiento')
    plt.plot(test_errors, label='Error de prueba')
    plt.xlabel('Iteración')
    plt.ylabel('Error cuadrático medio')
    plt.legend()
    plt.title('Error de entrenamiento y prueba vs iteraciones (Mini-Batch GD)')
    plt.show()

    return W

In [ ]:
mini_batch_gradient_descent(X_train, y_train_gd, X_test, y_test_gd, lr=0.1, epochs=100, batch_size=11)

# Regresion Logistica

In [ ]:
# y_train y y_test para la variale RainTomorrow
y_train_rl = df_train["RainTomorrow"]
y_test_rl = df_test["RainTomorrow"]

### Conjunto de entrenamiento

In [ ]:
logistic_model = LogisticRegression(random_state=42, class_weight='balanced')
logistic_model.fit(X_train, y_train_rl)
y_pred_train_rl = logistic_model.predict(X_train)
y_pred_test_rl = logistic_model.predict(X_test)
balanced_accuracy = balanced_accuracy_score(y_train_rl, y_pred_train_rl)

In [ ]:
# Calcular métricas
balanced_accuracy = balanced_accuracy_score(y_train_rl, y_pred_train_rl)
# accuracy_logreg = accuracy_score(y_train_rl, y_pred_train_rl)
confusion_matrix_logreg = confusion_matrix(y_train_rl, y_pred_train_rl)
classification_report_logreg = classification_report(y_train_rl, y_pred_train_rl)

# Imprimir métricas
print("Métricas para logreg:")
print(f'Accuracy balanceado: {balanced_accuracy}')
print("Matriz de confusión:")
print(confusion_matrix_logreg)
print("Reporte de clasificación:")
print(classification_report_logreg)
disp = ConfusionMatrixDisplay(confusion_matrix=confusion_matrix_logreg)
disp.plot()
plt.show()

En este caso de aplicacion, el interes se encuentra puesto en predecir si llovera o no, por lo que nos importa principalmente el caso donde se de un **verdadero positivo**, el modelo obtuvo sobre el **conjunto de entrenamiento** un total de 3559 casos en los que predijo que lloveria y llovio, por otra parte hay un total de 3569 casos de **falsos positivos**, predijo que lloveria y no llovio.

Observando la metrica **F1 Score** para los casos **positivos** es de 0.60, por lo que el modelo no es muy eficiente a la hora de predecir esta clase, mientras que para los casos **negativos** obtiene una metrica de 0.83 indicando que tiene mas facilidad para predecir dicha clase, esto puede que tenga que ver con el desbalanceo de clases que hay en el conjunto de datos, hay una mayor presencia de clases negativas que positivas, lo que tiene sentido con la realidad, debido a que generalmente son mas los dias en los que no llueve que los que si.

### Conjunto de test

In [ ]:
# Calcular métricas
balanced_accuracy = balanced_accuracy_score(y_test_rl, y_pred_test_rl)
# accuracy_logreg = accuracy_score(y_train_rl, y_pred_train_rl)
confusion_matrix_logreg = confusion_matrix(y_test_rl, y_pred_test_rl)
classification_report_logreg = classification_report(y_test_rl, y_pred_test_rl)

# Imprimir métricas
print("Métricas para logreg:")
print(f'Accuracy balanceado: {balanced_accuracy}')
print("Matriz de confusión:")
print(confusion_matrix_logreg)
print("Reporte de clasificación:")
print(classification_report_logreg)

disp = ConfusionMatrixDisplay(confusion_matrix=confusion_matrix_logreg)
disp.plot()
plt.show()

Para el **conjunto de test** vemos tambien valores muy parejos entre los **verdaderos positivos** y **falsos positivos** y obtuvimos los mismos valores que en el conjunto de entrenamiento en la metrica **F1 Score** para ambas clases.

### Curva ROC

In [ ]:
# Obtengo las probabilidades de predicción del modelo para los datos de entrenamiento y prueba
y_probs_train = logistic_model.predict_proba(X_train)[:, 1]
y_probs_test = logistic_model.predict_proba(X_test)[:, 1]

# Calculo la ROC y el AUC para los datos de entrenamiento y prueba
fpr_train, tpr_train, thresholds_train = roc_curve(y_train_rl, y_probs_train, pos_label= 1)
roc_auc_train = auc(fpr_train, tpr_train)

fpr_test, tpr_test, thresholds_test = roc_curve(y_test_rl, y_probs_test, pos_label= 1)
roc_auc_test = auc(fpr_test, tpr_test)

In [ ]:
# Grafico la curva ROC para los datos de entrenamiento y prueba
plt.figure(figsize=(8, 6))
plt.plot(fpr_train, tpr_train, color='darkorange', lw=2, label='Curva ROC para entrenamiento (AUC = %0.2f)' % roc_auc_train)
plt.plot(fpr_test, tpr_test, color='green', lw=2, label='Curva ROC para prueba (AUC = %0.2f)' % roc_auc_test)
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('Tasa de Falsos Positivos (FPR)')
plt.ylabel('Tasa de Verdaderos Positivos (TPR)')
plt.title('Curva ROC')
plt.legend(loc="lower right")
plt.show()

print(f"El área bajo la curva para entrenamiento es de: {roc_auc_train}")
print(f"El área bajo la curva para prueba es de: {roc_auc_test}")

In [ ]:
# Calcula la distancia euclidiana entre cada punto de la curva ROC y (0,1) para los datos de entrenamiento
distances_train = np.sqrt((1 - tpr_train)**2 + fpr_train**2)

# Encuentra el índice del punto que minimiza la distancia para los datos de entrenamiento
min_index_train = np.argmin(distances_train)

# Obtiene el umbral óptimo para los datos de entrenamiento
optimal_threshold_train = thresholds_train[min_index_train]

# Calcula la distancia euclidiana entre cada punto de la curva ROC y (0,1) para los datos de prueba
distances_test = np.sqrt((1 - tpr_test)**2 + fpr_test**2)

# Encuentra el índice del punto que minimiza la distancia para los datos de prueba
min_index_test = np.argmin(distances_test)

# Obtiene el umbral óptimo para los datos de prueba
optimal_threshold_test = thresholds_test[min_index_test]

print("Umbral óptimo para entrenamiento:", optimal_threshold_train)
print("Umbral óptimo para prueba:", optimal_threshold_test)

Optimizacion de hiperparametros para la **Regresion Logistica**

In [ ]:
from sklearn.model_selection import GridSearchCV

# Definir la cuadrícula de hiperparámetros a explorar
param_grid = {
    'penalty': [None, 'l1', 'l2'],
    'dual': [True, False],
    'C': [1, 1.5, 2, 2.5, 3],
    'fit_intercept': [True, False],
    'class_weight': [None, 'balanced', {0: 0.5, 1: 0.5}]
}

# Definir la cuadrícula de hiperparámetros a explorar
param_grid_2 = {
    'penalty': ['l1', 'l2'],  # Penalidades válidas
    'C': [0.1, 1, 10],  # Fuerza de regularización
    'class_weight': [None, 'balanced'],  # Pesos de clase
    'solver': ['liblinear', 'saga']  # Solver para optimización
}


# Inicializar la búsqueda en cuadrícula
grid_search = GridSearchCV(logreg, param_grid_2, cv=5)

# Realizar la búsqueda en cuadrícula
grid_search.fit(X_train, y_train_lg)

In [ ]:
# Obtener los mejores hiperparámetros
best_params = grid_search.best_params_
print("Mejores hiperparámetros:", best_params)

In [ ]:
logreg_opt = LogisticRegression(C=0.1, class_weight=None, penalty='l2', solver='saga')

In [ ]:
# Entrenar un modelo de regresión logística
logreg_opt.fit(X_train, y_train_lg)

# Realizar predicciones en el conjunto de prueba
y_pred_opt = logreg_opt.predict(X_test)

In [ ]:
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

# Calcular métricas
accuracy_logreg = accuracy_score(y_test_lg, y_pred_opt)
# accuracy_logreg_2D = accuracy_score(y_test_2D, y_pred_2D)

confusion_matrix_logreg = confusion_matrix(y_test_lg, y_pred_opt)
# confusion_matrix_logreg_2D = confusion_matrix(y_test_2D, y_pred_2D)

classification_report_logreg = classification_report(y_test_lg, y_pred_opt)
# classification_report_logreg_2D = classification_report(y_test_2D, y_pred_2D)

# Imprimir métricas
print("Métricas para logreg_opt:")
print(f'Precisión: {accuracy_logreg}')
print("Matriz de confusión:")
print(confusion_matrix_logreg)
print("Reporte de clasificación:")
print(classification_report_logreg)

disp = ConfusionMatrixDisplay(confusion_matrix=confusion_matrix_logreg)
disp.plot()
plt.show()

En este caso al usar GridSearch para optimizar hiperparametros no se observan cambios en las metricas, con respecto al modelo base.

In [ ]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_curve, roc_auc_score, auc
from sklearn import metrics
y_probs_opt = logreg_opt.predict_proba(X_test)[:, 1]

# Calculo la ROC y el AUC
fpr, tpr, thresholds = metrics.roc_curve(y_test_lg, y_probs_opt)
roc_auc = auc(fpr, tpr)

# Grafico la curva ROC
plt.figure(figsize=(6, 4))
plt.plot(fpr, tpr, color='darkorange', lw=2, label='Curva ROC (AUC = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('Tasa de Falsos Positivos (FPR)')
plt.ylabel('Tasa de Verdaderos Positivos (TPR)')
plt.title('Curva ROC')
plt.legend(loc="lower right")
plt.show()

Balanceo de clases con SMOTE

In [ ]:
# Aplica SMOTE para balancear las clases
from imblearn.over_sampling import SMOTE

smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X_train, y_train_lg)
grid_search.fit(X_resampled, y_resampled)
print("Mejores parámetros:", grid_search.best_params_)

In [ ]:
logreg_opt.fit(X_resampled, y_resampled)
# Realizar predicciones en el conjunto de prueba
y_pred_opt_smote = logreg_opt.predict(X_test)

In [ ]:
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

# Calcular métricas
accuracy_logreg = accuracy_score(y_test_lg, y_pred_opt_smote)
# accuracy_logreg_2D = accuracy_score(y_test_2D, y_pred_2D)

confusion_matrix_logreg = confusion_matrix(y_test_lg, y_pred_opt_smote)
# confusion_matrix_logreg_2D = confusion_matrix(y_test_2D, y_pred_2D)

classification_report_logreg = classification_report(y_test_lg, y_pred_opt_smote)
# classification_report_logreg_2D = classification_report(y_test_2D, y_pred_2D)

# Imprimir métricas
print("Métricas para logreg_opt:")
print(f'Precisión: {accuracy_logreg}')
print("Matriz de confusión:")
print(confusion_matrix_logreg)
print("Reporte de clasificación:")
print(classification_report_logreg)

disp = ConfusionMatrixDisplay(confusion_matrix=confusion_matrix_logreg)
disp.plot()
plt.show()

In [ ]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_curve, roc_auc_score, auc
from sklearn import metrics
y_pred_opt_smote = logreg_opt.predict_proba(X_test)[:, 1]

# Calculo la ROC y el AUC
fpr, tpr, thresholds = metrics.roc_curve(y_test_lg, y_pred_opt_smote)
roc_auc = auc(fpr, tpr)

# Grafico la curva ROC
plt.figure(figsize=(6, 4))
plt.plot(fpr, tpr, color='darkorange', lw=2, label='Curva ROC (AUC = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('Tasa de Falsos Positivos (FPR)')
plt.ylabel('Tasa de Verdaderos Positivos (TPR)')
plt.title('Curva ROC')
plt.legend(loc="lower right")
plt.show()

In [ ]:
from imblearn.over_sampling import RandomOverSampler, SMOTE
from sklearn.ensemble import RandomForestClassifier

In [ ]:
oversampler = RandomOverSampler(random_state=42)
X_resampled, y_resampled = oversampler.fit_resample(X_train, y_train_lg)

In [ ]:
len(y_resampled[y_resampled==1]), len(y_resampled[y_resampled==0])

In [ ]:
logreg.fit(X_resampled, y_resampled)
y_pred = logreg.predict(X_test)

print("Resultados regresión logística con Oversampling:\n")
print(classification_report(y_test_lg, y_pred))
print(confusion_matrix(y_test_lg, y_pred))

In [ ]:
# Obtengo las probabilidades de predicción del modelo
y_probs = logreg.predict_proba(X_test)[:, 1]

# Calculo la ROC y el AUC
fpr, tpr, thresholds = roc_curve(y_test_lg, y_probs)
roc_auc = auc(fpr, tpr)

In [ ]:
# Grafico la curva ROC
plt.figure(figsize=(6, 4))
plt.plot(fpr, tpr, color='darkorange', lw=2, label='Curva ROC (AUC = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('Tasa de Falsos Positivos (FPR)')
plt.ylabel('Tasa de Verdaderos Positivos (TPR)')
plt.title('Curva ROC')
plt.legend(loc="lower right")
plt.show()

# Ejercicio 6 - Modelos Base

### Modelo Base de Regresion

In [ ]:
X_train_rl = X_train[["Rainfall", "RainToday"]]
X_test_rl = X_test[["Rainfall", "RainToday"]]

In [ ]:
model_rl = LinearRegression()

model_rl.fit(X_train_rl, y_train)

y_pred = model_rl.predict(X_test_rl)

coefficients = model_rl.coef_
intercept = model_rl.intercept_
print("Coefficients:", coefficients)
print("Intercept:", intercept)

# MSE: Error Cuadratico Medio
mse = mean_squared_error(y_test, y_pred)
# R^2
r2 = r2_score(y_test, y_pred)
# MAE: Error Absoluto Medio
mae = mean_absolute_error(y_test, y_pred)
# RMSE: Raíz del Error Cuadrático Medio
rmse = np.sqrt(mean_squared_error(y_test, y_pred))

print(f"\nR^2: {r2}\n")
print(f"MSE(Error Cuadratico Medio): {mse}\n")
print(f"MAE(Error Absoluto Medio): {mae}\n")
print(f"RMSE(Raíz del Error Cuadrático Medio): {rmse}\n")

El coeficiente **R²** es de 0.084 lo que indica que el modelo no tiene una gran capacidad para explicar la variabilidad de los datos. El modelo no captura adecuadamente las relaciones entre las variables predictoras y la variable objetivo.

El **MSE** con un valor de 0.9116 nos demuestra que las predicciones del modelo estan bastate dispersas respecto a los valores observados, se puede decir que su ajuste es bastante malo.

**Conclusion**: Las metricas obtenidas nos sugieren que las predicciones del modelo no son precisas y tienen un error significativo. Esto podría ser porque el modelo es demasiado simple.

### Modelo Base de Clasificacion

In [ ]:
from sklearn.dummy import DummyClassifier
from sklearn.metrics import accuracy_score

In [ ]:
dummy_random = DummyClassifier(strategy='uniform')
dummy_random.fit(X_train_rl, y_train_rl)
y_pred_random = dummy_random.predict(X_test_rl)

accuracy_random = accuracy_score(y_test_rl, y_pred_random)
print("Accuracy del clasificador aleatorio:", accuracy_random)


# Ejercicio 7

### Validacion Cruzada K-Folds

In [ ]:
from sklearn.model_selection import cross_val_score, StratifiedKFold, KFold, LeaveOneOut
from sklearn.ensemble import RandomForestClassifier

In [ ]:
clf = RandomForestClassifier(n_estimators=50, random_state=42)

In [ ]:
X_train_k, x_val_k, y_train_k, y_val_k = train_test_split(X_train_rl, y_train_rl, test_size=0.2, random_state=42)

In [ ]:
X_train_k

In [ ]:
x_val_k

In [ ]:
y_train_k

In [ ]:
y_val_k

In [ ]:
cv_strategies = [
    ("KFold", KFold(n_splits=5, shuffle=True, random_state=42))
]

results = {}
for name, cv in cv_strategies:
    scores = cross_val_score(clf, x_val_k, y_val_k, cv=cv)
    results[name] = scores

In [ ]:
plt.figure(figsize=(10, 6))
for name, scores in results.items():
    plt.plot(range(1, len(scores) + 1), scores, marker='o', label=name)

plt.xlabel("Fold")
plt.ylabel("Accuracy")
plt.title("Estrategias de validación cruzada")
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
results['KFold']

In [ ]:
for strategy in cv_strategies:
  print('Media para la estrategia', strategy[0],':',results['KFold'].mean())
  print('Desvío estándar para la estrategia', strategy[0],':',results['KFold'].std())

# SHAP

### Explicabilidad Local

In [ ]:
feature_names = X_train.columns.values
explainer = shap.LinearExplainer(logistic_model, X_train, feature_names=feature_names)

In [ ]:
shap_values = explainer.shap_values(X_test)
explainer.expected_value

In [ ]:
explanation = shap.Explanation(values=shap_values[0], base_values=explainer.expected_value, feature_names=feature_names)
shap.plots.bar(explanation, max_display=21)

A nivel local, **WindSpeed_Diference** y **Cloud_Difference** son las variables que mas impactan sobre la prediccion de **RainTomorrow**

### Explicabilidad global

In [ ]:
explanation = shap.Explanation(values=shap_values, base_values=explainer.expected_value, feature_names=feature_names, data=X_test_rl)

In [ ]:
shap.plots.bar(explanation, max_display=21)

Las variables de mayor importancia a nivel global son **Sunshine** y **WindSpeed_Difference**, esta ultima tambien observada en la explicabilidad local.

# Redes Neuronales

In [ ]:
import tensorflow as tf

In [ ]:
class NeuralNetworkTensorFlow:
    """
        Este es un modelo simple con TensorFlow para resolver el mismo problema. 
        En esta clase, (1) se construye el modelo.
        (2) Se define como se fitea el modelo
        (3) Y como se hacen las predicciones.
    """
    def __init__(self):
        self.model = self.build_model()

    def build_model(self):
        """
            Construye el modelo
            Para construir el modelo es necesario una arquitectura, un optimizador y una función de pérdida.
            La arquitectura se construye con el método Sequential, que basicamente lo que hace es colocar 
            secuencialmente las capas que uno desea.
            Las capas "Dense" son las fully connected dadas en clase.
            Se agrega una capa oculta que recibe un input de tamaño 2,
            y una capa de salida de regresión (una única neurona)
            En todos los casos se define una sigmoidea como función de activación (prueben otras!)

            El optimizador y la función de pérdida se especifican dentro de un compilador.

            Con este método, lo que se devuelve es el modelo sin entrenar, sería equivalente a escribir LinearRegression() 
            en el caso de la regresión lineal.
        """

        model = tf.keras.Sequential([
            tf.keras.layers.Dense(2, activation='sigmoid', input_shape=(20,)),
            tf.keras.layers.Dense(1, activation='sigmoid')
        ])
        model.compile(optimizer='adam', loss='mean_squared_error')
        ### imprimimos la cantidad de parámetros a modo de ejemplo
        print("n° de parámetros:", model.count_params())
        return model
    
    def fit(self, X, y, lr=0.1, epochs=20000):
        ### esta es la función donde se entrena el modelo, fijarse que hay un learning rate e iteraciones.
        ### la función que fitea devuelve una historia de pérdida, que vamos a guardar para graficar la evolución.
        X = np.array(X)
        y = np.array(y)
        history = self.model.fit(X, y, epochs=epochs, verbose=0)
        return history.history['loss']

    def predict(self, X):
        X = np.array(X)
        predictions = self.model.predict(X)
        return predictions

### Regresion Lineal

In [ ]:
print(f'MSE: {mse}')

In [ ]:
nn_tensorflow = NeuralNetworkTensorFlow()
loss_history = nn_tensorflow.fit(X_train, y_train, lr=0.1, epochs=100)

In [ ]:
plt.plot(loss_history, label='NN')
plt.axhline(mse, color='red', label='linear',linestyle = '-')
plt.legend()
plt.show()

#### Optimizacion de hiperparametros

In [ ]:
import optuna
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [ ]:
def objective(trial):

    num_layers = trial.suggest_int('num_layers', 1, 3)
    model = Sequential()

    for i in range(num_layers):
        num_units = trial.suggest_int(f'n_units_layer_{i}', 4, 128) # la cantidad de neuronas de cada capa tambien se puede pasar como hiperparámetro
        # activations = trial.suggest_categorical(f'')
        model.add(Dense(num_units, activation='sigmoid')) # capas densas con activacion ReLU

    # capa de salida
    model.add(Dense(1)) # 3 son las clases de salida

    # compilar
    model.compile(optimizer='Adagrad', loss='categorical_crossentropy', metrics=['mse'])

    epochs = trial.suggest_int('epochs', 5, 100)

    # entrenar
    model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=5, batch_size=32, verbose=0)

    # evaluar
    score = model.evaluate(X_test, y_test, verbose=0)
    return score[1]

In [ ]:
# crear un estudio de Optuna
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=50)

# obtener los mejores hiperparámetros
best_params = study.best_params
print("Best parámetros encontrados:", best_params)

Al utilizar optuna, obtuve diferentes resultados, voy a quedarme con el siguiente.
Mejores parámetros encontrados: {'num_layers': 3, 'n_units_layer_0': 54, 'n_units_layer_1': 95, 'n_units_layer_2': 92, 'epochs': 29}

Nueva clase de nn_tensorflow con hiperparametros ajustados

In [ ]:
import tensorflow as tf
import numpy as np

class NeuralNetworkTensorFlowOpt:
    def __init__(self):
        self.model = self.build_model()

    def build_model(self):
        num_layers = 3
        n_units_layer_0 = 54
        n_units_layer_1 = 95

        model = tf.keras.Sequential()
        model.add(tf.keras.layers.Dense(n_units_layer_0, activation='sigmoid', input_shape=(20,)))

        for _ in range(1, num_layers):
            model.add(tf.keras.layers.Dense(n_units_layer_1, activation='sigmoid'))

        model.add(tf.keras.layers.Dense(1, activation='sigmoid'))

        model.compile(optimizer='adam', loss='mean_squared_error')
        
        ### imprimimos la cantidad de parámetros a modo de ejemplo
        print("n° de parámetros:", model.count_params())
        return model
    
    def fit(self, X, y, lr=0.1, epochs=29):
        ### esta es la función donde se entrena el modelo, fijarse que hay un learning rate e iteraciones.
        ### la función que fitea devuelve una historia de pérdida, que vamos a guardar para graficar la evolución.
        X = np.array(X)
        y = np.array(y)
        history = self.model.fit(X, y, epochs=epochs, verbose=0)
        return history.history['loss']

    def predict(self, X):
        X = np.array(X)
        predictions = self.model.predict(X)
        return predictions


In [ ]:
nn_tensorflowOpt = NeuralNetworkTensorFlowOpt()
loss_history_opt = nn_tensorflowOpt.fit(X_train, y_train, lr=0.1, epochs=29)

In [ ]:
plt.plot(loss_history_opt, label='NN')
plt.axhline(mse, color='red', label='linear',linestyle = '-')
plt.legend()
plt.show()

Puedo apreciar que en esta ocacion se logra obtener un valor de MSE menor al obtenido en el modelo de Regresion Lineal implementado anteriormente.

### Regresion Logistica

In [ ]:
print(f'Precisión obtenida en el modelo implemtado anteriormente: {accuracy_logreg}')

Utilizo el y_train_rl generado en la seccion de Regresion Logistica ya que tiene valores 1 y 0 correspondientes a cada clase.

Defino una nueva funcion objetivo adaptandola a un modelo de clasificacion.

In [ ]:
import optuna
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split

In [ ]:
def objective(trial):

    num_layers = trial.suggest_int('num_layers', 1, 3)
    model = Sequential()

    for i in range(num_layers):
        num_units = trial.suggest_int(f'n_units_layer_{i}', 4, 128) # la cantidad de neuronas de cada capa tambien se puede pasar como hiperparámetro
        # activations = trial.suggest_categorical(f'')
        model.add(Dense(num_units, activation='relu')) # capas densas con activacion ReLU

    # capa de salida
    model.add(Dense(1, activation='sigmoid')) # 2 son las clases de salida

    # compilar
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

    # Sugerir el número de epochs como hiperparámetro
    epochs = trial.suggest_int('epochs', 5, 50)

    # entrenar
    model.fit(X_train, y_train_lg, validation_data=(X_test, y_test_lg), epochs=5, batch_size=32, verbose=0)

    # evaluar
    score = model.evaluate(X_test, y_test_lg, verbose=0)
    return score[1]

In [ ]:
# crear un estudio de Optuna
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=50)

# obtener los mejores hiperparámetros
best_params = study.best_params
print("Best parámetros encontrados:", best_params)

Mejores parámetros encontrados: {'num_layers': 3, 'n_units_layer_0': 41, 'n_units_layer_1': 24, 'n_units_layer_2': 83, 'epochs': 39}

In [ ]:
# modelo de red neuronal
model = Sequential()
model.add(Dense(90, activation='relu'))  # Capa oculta con 38 neuronas y función de activación ReLU
model.add(Dense(90, activation='relu'))  # Capa oculta con 38 neuronas y función de activación ReLU
# model.add(Dense(38, activation='relu'))  # Capa oculta con 38 neuronas y función de activación ReLU
model.add(Dense(1, activation='sigmoid'))  # Capa de salida con 1 neurona  y función de activación sigmoid

In [ ]:
# modelo de red neuronal
model = Sequential()
model.add(Dense(38, activation='relu'))  # Capa oculta con 38 neuronas y función de activación ReLU
model.add(Dense(38, activation='relu'))  # Capa oculta con 38 neuronas y función de activación ReLU
model.add(Dense(38, activation='relu'))  # Capa oculta con 38 neuronas y función de activación ReLU
model.add(Dense(1, activation='sigmoid'))  # Capa de salida con 1 neurona  y función de activación sigmoid

In [ ]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
model.fit(X_train, y_train_rl, epochs=48, batch_size=32, validation_data=(X_test, y_test_rl))

In [ ]:
# Hacer predicciones
y_pred2_rn = model.predict(X_test)
y_pred = (y_pred2_rn > 0.5).astype(int)  # Clasificación binaria

# Métricas para el modelo sin optimización
f1_score_rn = f1_score(y_test_rl, y_pred, average='weighted')

print(f"F1 Score del modelo: {f1_score_rn}")

In [ ]:
# Calcular métricas
balanced_accuracy = balanced_accuracy_score(y_test_rl, y_pred)
# accuracy_logreg = accuracy_score(y_train_rl, y_pred_train_rl)
confusion_matrix_logreg = confusion_matrix(y_test_rl, y_pred)
classification_report_logreg = classification_report(y_test_rl, y_pred)

# Imprimir métricas
print("Métricas para logreg:")
print(f'Accuracy balanceado: {balanced_accuracy}')
print("Matriz de confusión:")
print(confusion_matrix_logreg)
print("Reporte de clasificación:")
print(classification_report_logreg)
disp = ConfusionMatrixDisplay(confusion_matrix=confusion_matrix_logreg)
disp.plot()
plt.show()

Usando redes neuronales para clasificar, obtuve valores muy similares a los obtenidos en el modelo implementado anteriormente, los valores de **F1 Score** para cada clase se encuentran casi invariantes.

### SHAP

#### Explicabilidad Local

In [ ]:
background = X_train.sample(100)

In [ ]:
feature_names = X_train.columns.values
explainer = shap.KernelExplainer(model, background)

In [ ]:
shap_values = explainer.shap_values(instance_to_explain)
explainer.expected_value

In [ ]:
explanation = shap.Explanation(values=shap_values[0], base_values=explainer.expected_value, feature_names=feature_names)
shap.plots.bar(explanation, max_display=21)

A nivel local, **WindSpeed_Diference** y **Cloud_Difference** son las variables que mas impactan sobre la prediccion de **RainTomorrow**

#### Explicabilidad global

In [ ]:
explanation = shap.Explanation(values=shap_values, base_values=explainer.expected_value, feature_names=feature_names, data=X_test_rl)

In [ ]:
shap.plots.bar(explanation, max_display=21)

Las variables de mayor importancia a nivel global son **Sunshine** y **WindSpeed_Difference**, esta ultima tambien observada en la explicabilidad local.

In [ ]:
# Crear un conjunto de muestras de fondo
background = X_train.sample(100)  
# Crear el objeto explainer SHAP utilizando KernelExplainer
explainer = shap.KernelExplainer(model, background)
instance_to_explain = X_test.iloc[0:1].values  # Seleccionar la línea 0
# Calcular los valores SHAP para los datos de prueba
shap_values = explainer.shap_values(instance_to_explain)

In [ ]:
# Obtener el valor esperado (base value)
expected_value = explainer.expected_value
# Hacer predicciones
index = 0  # índice de la instancia que queremos explicar
predicted_proba = model.predict(X_test)[index]  # Predicción de probabilidad
predicted_class = np.argmax(predicted_proba)  # Clase predicha

In [ ]:
# Crear la explicación para la instancia específica
explanation = shap.Explanation(values=shap_values[0], base_values=expected_value, feature_names=feature_names)

# Visualizar la explicación
shap.initjs()
shap.plots.bar(explanation, max_display=len(feature_names))